In [6]:
%load_ext autoreload
%autoreload 2
import json
import os
import numpy as np
from collections import Counter
import sklearn
from sklearn import preprocessing
import reptile
import reptile.model
import reptile.train
import reptile.eval
import reptile.reptile
import tensorflow as tf

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
glove_file = 'glove.6B/glove.6B.50d.txt'
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [68]:
cur_dir = '/Users/crazydonkey/projects/meta-start/'
data_fn = 'splitted_data_pos_3.json'
with open(os.path.join(cur_dir, 'data/preprocess_data/', data_fn), 'r') as f:
    splitted_data = json.load(f)

In [69]:
all_cats = list(splitted_data['train'].keys()) + list(splitted_data['dev'].keys()) + list(splitted_data['test'].keys())

In [70]:
# all_cats = [c.lower() for c in all_cats]
# all_tokenized_cats = []
# for cat in all_cats:
#     all_tokenized_cats.append(cat.split(' '))
# all_tokenized_cats

In [71]:
embedding_dim = model['man'].shape[0]
embedding_dim

50

In [72]:
np.random.seed(0)
unknown_vec = np.random.randn(embedding_dim)
unknown_vec = unknown_vec / np.linalg.norm(unknown_vec)
unknown_vec

array([ 0.21994376,  0.04989199,  0.12203001,  0.27939675,  0.23284895,
       -0.12184796,  0.11845795, -0.01887136, -0.01286943,  0.05119382,
        0.01795949,  0.18132024,  0.09488692,  0.01517056,  0.0553413 ,
        0.04160284,  0.18628323, -0.02557933,  0.03903359, -0.10648949,
       -0.31830926,  0.0814938 ,  0.10777875, -0.09253386,  0.28299522,
       -0.18133173,  0.00570522, -0.02333826,  0.19110841,  0.18320108,
        0.019319  ,  0.04714967, -0.11068999, -0.24696763, -0.04337803,
        0.01949374,  0.15339384,  0.14991389, -0.04829228, -0.0376914 ,
       -0.1307346 , -0.17704922, -0.21273943,  0.24322458, -0.06354393,
       -0.05461953, -0.15619975,  0.09693825, -0.20122236, -0.02652467])

In [73]:
cat_embeddings = {}
for cat in all_cats:
    tks = cat.lower().split(' ')
    sum_array = np.zeros(embedding_dim)
    for tk in tks:
        if tk in model:
            tk_vec = model[tk]
            tk_vec = tk_vec / np.linalg.norm(tk_vec)
            sum_array = sum_array + tk_vec
        else:
            sum_array += unknown_vec
    cat_embeddings[cat] = list(sum_array / len(tks)) 

In [74]:
cur_dir

'/Users/crazydonkey/projects/meta-start/'

In [75]:
with open(cur_dir + 'data/preprocess_data/category_embeddings.json', 'w') as f:
    json.dump(cat_embeddings, f)

In [76]:
with open(cur_dir + 'data/preprocess_data/category_embeddings.json', 'r') as f:
    cat_embeddings = json.load(f)

for k, v in cat_embeddings.items():
    cat_embeddings[k] = np.array(v)

In [78]:
cat_embeddings['Wireless']
len(cat_embeddings)

389

In [53]:
# for cat in all_tokenized_cats:
#     print(len(cat))

In [65]:
np.concatenate([raw_data_mat, np.tile(np.reshape(cat_embeddings['Wireless'], (1, 50)), [32009, 1])], axis=1)

array([[ 2.00000000e+06,  1.00000000e+00,  1.00000000e+00, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02],
       [ 4.12500000e+04,  1.00000000e+00,  1.00000000e+00, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02],
       [ 3.36000000e+07,  4.00000000e+00,  1.40000000e+01, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02],
       ...,
       [ 1.58730100e+06,  1.00000000e+00,  1.00000000e+00, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02],
       [ 1.14304000e+05,  4.00000000e+00,  3.00000000e+00, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02],
       [ 1.81920000e+04,  1.00000000e+00,  1.00000000e+00, ...,
         2.06713229e-02, -1.45791888e-01,  9.94145796e-02]])

In [54]:
splitted_data['companies']['1']

{'funding_total_usd': 41250.0,
 'country_code': 'HKG',
 'funding_rounds': 1,
 'Num_of_investor': 1,
 'funding_duration': 0,
 'first_funding_at_UTC': 16252,
 'last_funding_at_UTC': 16252,
 'label': 0}

In [55]:
len(splitted_data['companies'].keys())

32009

In [56]:
company_int_keys = sorted([int(s) for s in splitted_data['companies'].keys()])

In [57]:
company_col_dict = {}
company_int_key_to_pos = {}
for k in splitted_data['companies']['1'].keys():
    company_col_dict[k] = []
for i, key in enumerate(company_int_keys):
    company_info = splitted_data['companies'][str(key)]
    for col, v in company_info.items():
        company_col_dict[col].append(v)
        company_int_key_to_pos[key] = i

feature_names = []
col_arrays = []
for k, v in company_col_dict.items():
    if not isinstance(v[0], str):
        feature_names.append(k)
        col_arrays.append(np.array(company_col_dict[k]))
print(feature_names)
company_mat = np.vstack(col_arrays).transpose()
raw_data_mat, label_mat = np.split(company_mat, [6, ], axis=1)
label_mat = label_mat.reshape(-1)

['funding_total_usd', 'funding_rounds', 'Num_of_investor', 'funding_duration', 'first_funding_at_UTC', 'last_funding_at_UTC', 'label']


In [58]:
print(raw_data_mat.shape)
print(label_mat.shape)

(32009, 6)
(32009,)


In [59]:
def update_to_new_id(old_dict):
    new_dict = {}
    for k, v in old_dict.items():
        new_dict[k] = [company_int_key_to_pos[idx] for idx in old_dict[k]]
    return new_dict

train_cat_dict = update_to_new_id(splitted_data['train'])
dev_cat_dict = update_to_new_id(splitted_data['dev'])
test_cat_dict = update_to_new_id(splitted_data['test'])

In [60]:
def collect_ids(cat_dict):
    total_ids = set()
    for k, v in cat_dict.items():
        total_ids = total_ids.union(set(v))
    return total_ids

train_ids = collect_ids(train_cat_dict)
print('train: {} cats, {} unique companies, positive: {}'.format(
    len(train_cat_dict.keys()), len(train_ids), np.mean(label_mat[list(train_ids)])))
dev_ids = collect_ids(dev_cat_dict)
print('dev: {} cats, {} unique companies, positive: {}'.format(
    len(dev_cat_dict.keys()), len(dev_ids), np.mean(label_mat[list(dev_ids)])))
test_ids = collect_ids(test_cat_dict)
print('test: {} cats, {} unique companies, positive: {}'.format(
    len(test_cat_dict.keys()), len(test_ids), np.mean(label_mat[list(test_ids)])))

train: 178 cats, 28045 unique companies, positive: 0.1491888037083259
dev: 50 cats, 2423 unique companies, positive: 0.15270326042096574
test: 50 cats, 1541 unique companies, positive: 0.20571057754704739


In [61]:
for cat, ids in train_cat_dict.items():
    print(np.sum(label_mat[ids]), np.mean(label_mat[ids]))

41.0 0.21808510638297873
28.0 0.11475409836065574
7.0 0.0660377358490566
14.0 0.13861386138613863
16.0 0.22857142857142856
5.0 0.0847457627118644
10.0 0.049019607843137254
37.0 0.14453125
154.0 0.15859938208032956
19.0 0.14615384615384616
29.0 0.17058823529411765
13.0 0.20634920634920634
29.0 0.20567375886524822
7.0 0.07216494845360824
7.0 0.07777777777777778
13.0 0.08387096774193549
165.0 0.16751269035532995
34.0 0.15178571428571427
15.0 0.16853932584269662
6.0 0.07058823529411765
71.0 0.16904761904761906
9.0 0.13846153846153847
9.0 0.15789473684210525
6.0 0.07692307692307693
38.0 0.125
8.0 0.07407407407407407
80.0 0.18099547511312217
6.0 0.03076923076923077
17.0 0.10365853658536585
5.0 0.07936507936507936
12.0 0.0851063829787234
17.0 0.09289617486338798
10.0 0.1282051282051282
154.0 0.13106382978723405
8.0 0.1
24.0 0.08888888888888889
34.0 0.2982456140350877
42.0 0.19718309859154928
23.0 0.07232704402515723
54.0 0.17532467532467533
35.0 0.12195121951219512
104.0 0.13248407643312102
5

In [62]:
for cat, ids in test_cat_dict.items():
    print(np.sum(label_mat[ids]), np.mean(label_mat[ids]))

5.0 0.1724137931034483
10.0 0.37037037037037035
6.0 0.16666666666666666
7.0 0.18421052631578946
7.0 0.1794871794871795
10.0 0.2777777777777778
7.0 0.2692307692307692
5.0 0.29411764705882354
7.0 0.2
9.0 0.20930232558139536
6.0 0.13953488372093023
5.0 0.15151515151515152
6.0 0.16666666666666666
7.0 0.1794871794871795
10.0 0.3333333333333333
5.0 0.13513513513513514
6.0 0.3
5.0 0.35714285714285715
6.0 0.16216216216216217
5.0 0.13513513513513514
7.0 0.19444444444444445
15.0 0.42857142857142855
6.0 0.18181818181818182
8.0 0.2
5.0 0.13513513513513514
5.0 0.19230769230769232
8.0 0.1951219512195122
5.0 0.15625
5.0 0.2631578947368421
9.0 0.20930232558139536
6.0 0.3333333333333333
6.0 0.2857142857142857
7.0 0.16666666666666666
10.0 0.29411764705882354
6.0 0.17647058823529413
5.0 0.1724137931034483
8.0 0.21052631578947367
5.0 0.16666666666666666
5.0 0.2777777777777778
6.0 0.24
5.0 0.12195121951219512
7.0 0.1794871794871795
5.0 0.20833333333333334
6.0 0.2222222222222222
5.0 0.18518518518518517
7.0 

In [122]:
np.sum(label_mat[list(test_ids)])

317.0

In [123]:
# test_cat_dict['Humanitarian']

In [124]:
# splitted_data['test']['Humanitarian']

In [125]:
print(splitted_data['companies']['29337'])
print(company_mat[29314])

{'funding_total_usd': 4700000.0, 'country_code': 'USA', 'funding_rounds': 2, 'Num_of_investor': 2, 'funding_duration': 1099, 'first_funding_at_UTC': 13083, 'last_funding_at_UTC': 14182, 'label': 1}
[1.1280346e+07 1.0000000e+00 1.0000000e+00 0.0000000e+00 1.5771000e+04
 1.5771000e+04 0.0000000e+00]


In [126]:
train_data = raw_data_mat[list(train_ids)]
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
data_mat = scaler.transform(raw_data_mat)

In [136]:
class Dataset:
    def __init__(self, cat_dict, data_mat, label_mat):
        self.cat_dict = cat_dict
        self.data_mat = data_mat
        self.label_mat = label_mat
    
#     def sample_mini_dataset(self, n_examples):
#         cats = list(self.cat_dict.keys())
#         selected_cat = np.random.choice(cats)
#         selected_ids = np.random.choice(self.cat_dict[selected_cat], n_examples, replace=False)
#         return list(zip(self.data_mat[selected_ids], self.label_mat[selected_ids]))

    def sample_mini_dataset(self, n_example_per_class):
        cats = list(self.cat_dict.keys())
        selected_cat = np.random.choice(cats)
        all_ids = self.cat_dict[selected_cat]
        pos_ids = [x for x in all_ids if self.label_mat[x] == 1]
        neg_ids = [x for x in all_ids if self.label_mat[x] == 0]
        if len(pos_ids) < n_example_per_class:
            print(n_example_per_class)
            print(pos_ids)
        if len(neg_ids) < n_example_per_class:
            print(neg_ids)
        selected_pos_ids = np.random.choice(pos_ids, n_example_per_class, replace=False)
        selected_neg_ids = np.random.choice(neg_ids, n_example_per_class, replace=False)
        return (list(zip(self.data_mat[selected_pos_ids], self.label_mat[selected_pos_ids])) +
                list(zip(self.data_mat[selected_neg_ids], self.label_mat[selected_neg_ids])))

In [137]:
train_set = Dataset(train_cat_dict, data_mat, label_mat)
dev_set = Dataset(dev_cat_dict, data_mat, label_mat)
test_set = Dataset(test_cat_dict, data_mat, label_mat)

In [138]:
train_set.sample_mini_dataset(4)

[(array([ 6.33219326, -0.7211086 , -0.6430681 , -0.67775113, -1.30050225,
         -1.88267018]), 1.0),
 (array([-0.13457504, -0.10749459,  0.47567253, -0.17286667, -1.27776558,
         -1.51874439]), 1.0),
 (array([ 0.39967572, -0.10749459, -0.27015455,  0.30923204, -2.95294435,
         -3.03402833]), 1.0),
 (array([-0.11188235, -0.10749459, -0.45661132, -0.4822733 ,  0.02335838,
         -0.29814372]), 1.0),
 (array([-0.11901434, -0.7211086 , -0.08369778, -0.67775113, -2.46740654,
         -3.16365677]), 0.0),
 (array([-0.15094619, -0.7211086 , -0.27015455, -0.67775113,  1.02377162,
          0.6688361 ]), 0.0),
 (array([ 0.34861716,  3.57418952,  5.13709182,  1.66198661, -0.20620858,
          0.88944598]), 0.0),
 (array([ 0.21519631,  1.11973345,  0.6621293 ,  2.52544698, -1.28876719,
          0.28075594]), 0.0)]

In [142]:
model = reptile.model.ConstantPredictionClassifier(2, 6)

inner_iters=1
eval_inner_iters=1
num_shots=4
with tf.Session() as sess:
    print('Training...')
    reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                        meta_iters=1, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
    print('Evaluating...')
    print('Train accuracy: ' + str(reptile.eval.evaluate(
        sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))
    print('Test accuracy: ' + str(reptile.eval.evaluate(
        sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))

Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
Evaluating...
Train accuracy: 0.5
Test accuracy: 0.5


In [159]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    inner_iters=1
    eval_inner_iters=10
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=1, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.405
Test accuracy: 0.395
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.525
Test accuracy: 0.49
Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
Evaluating...
Train accuracy: 0.575
Test accuracy: 0.545
Training...
correct:  1
correct:  2
batch 0: train=1.000000 test=2.000000
Evaluating...
Train accuracy: 0.605
Test accuracy: 0.635
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.595
Test accuracy: 0.6
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
Evaluating...
Train accuracy: 0.54
Test accuracy: 0.43
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
Evaluating...
Train accuracy: 0.665
Test accuracy: 0.645
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
Evaluating...


In [160]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=10
    inner_iters=10
    eval_inner_iters=10
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.615
Test accuracy: 0.58
Training...
correct:  1
correct:  0
batch 0: train=1.000000 test=0.000000
Evaluating...
Train accuracy: 0.53
Test accuracy: 0.535
Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
Evaluating...
Train accuracy: 0.58
Test accuracy: 0.585
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.42
Test accuracy: 0.405
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
Evaluating...
Train accuracy: 0.62
Test accuracy: 0.62
Training...
correct:  1
correct:  2
batch 0: train=1.000000 test=2.000000
Evaluating...
Train accuracy: 0.605
Test accuracy: 0.625
Training...
correct:  1
correct:  0
batch 0: train=1.000000 test=0.000000
Evaluating...
Train accuracy: 0.66
Test accuracy: 0.62
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
Evaluating...
Tra

In [161]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=50
    inner_iters=10
    eval_inner_iters=10
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  2
correct:  0
batch 0: train=2.000000 test=0.000000
correct:  1
correct:  0
batch 10: train=1.000000 test=0.000000
correct:  1
correct:  2
batch 20: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 30: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 40: train=1.000000 test=2.000000
Evaluating...
Train accuracy: 0.715
Test accuracy: 0.635
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 10: train=2.000000 test=2.000000
correct:  1
correct:  0
batch 20: train=1.000000 test=0.000000
correct:  2
correct:  1
batch 30: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 40: train=2.000000 test=1.000000
Evaluating...
Train accuracy: 0.69
Test accuracy: 0.645
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 10: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 20: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 30: t

In [162]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=100
    inner_iters=10
    eval_inner_iters=10
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 10: train=1.000000 test=1.000000
correct:  1
correct:  0
batch 20: train=1.000000 test=0.000000
correct:  1
correct:  0
batch 30: train=1.000000 test=0.000000
correct:  2
correct:  2
batch 40: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 50: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 60: train=1.000000 test=2.000000
correct:  2
correct:  0
batch 70: train=2.000000 test=0.000000
correct:  2
correct:  1
batch 80: train=2.000000 test=1.000000
correct:  2
correct:  0
batch 90: train=2.000000 test=0.000000
Evaluating...
Train accuracy: 0.695
Test accuracy: 0.605
Training...
correct:  2
correct:  1
batch 0: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 10: train=2.000000 test=2.000000
correct:  0
correct:  1
batch 20: train=0.000000 test=1.000000
correct:  2
correct:  2
batch 30: train=2.000000 test=2.000000
correct:  0
correct:  1
batch 40: train

In [163]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=1000
    inner_iters=10
    eval_inner_iters=10
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
correct:  0
correct:  1
batch 10: train=0.000000 test=1.000000
correct:  1
correct:  1
batch 20: train=1.000000 test=1.000000
correct:  0
correct:  1
batch 30: train=0.000000 test=1.000000
correct:  1
correct:  1
batch 40: train=1.000000 test=1.000000
correct:  0
correct:  2
batch 50: train=0.000000 test=2.000000
correct:  1
correct:  0
batch 60: train=1.000000 test=0.000000
correct:  2
correct:  1
batch 70: train=2.000000 test=1.000000
correct:  1
correct:  0
batch 80: train=1.000000 test=0.000000
correct:  0
correct:  1
batch 90: train=0.000000 test=1.000000
correct:  1
correct:  1
batch 100: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 110: train=2.000000 test=1.000000
correct:  0
correct:  1
batch 120: train=0.000000 test=1.000000
correct:  0
correct:  0
batch 130: train=0.000000 test=0.000000
correct:  1
correct:  1
batch 140: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 150: trai

correct:  2
correct:  2
batch 280: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 290: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 300: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 310: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 320: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 330: train=2.000000 test=1.000000
correct:  1
correct:  0
batch 340: train=1.000000 test=0.000000
correct:  2
correct:  2
batch 350: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 360: train=1.000000 test=1.000000
correct:  1
correct:  2
batch 370: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 380: train=1.000000 test=2.000000
correct:  2
correct:  0
batch 390: train=2.000000 test=0.000000
correct:  1
correct:  1
batch 400: train=1.000000 test=1.000000
correct:  1
correct:  2
batch 410: train=1.000000 test=2.000000
correct:  1
correct:  1
batch 420: train=1.000000 test=1.000000
correct:  1
correct:  0
batch 430: train

correct:  2
correct:  1
batch 560: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 570: train=1.000000 test=2.000000
correct:  1
correct:  1
batch 580: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 590: train=1.000000 test=1.000000
correct:  1
correct:  2
batch 600: train=1.000000 test=2.000000
correct:  0
correct:  2
batch 610: train=0.000000 test=2.000000
correct:  2
correct:  1
batch 620: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 630: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 640: train=2.000000 test=2.000000
correct:  1
correct:  0
batch 650: train=1.000000 test=0.000000
correct:  2
correct:  1
batch 660: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 670: train=2.000000 test=1.000000
correct:  0
correct:  0
batch 680: train=0.000000 test=0.000000
correct:  1
correct:  2
batch 690: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 700: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 710: train

correct:  2
correct:  2
batch 840: train=2.000000 test=2.000000
correct:  0
correct:  2
batch 850: train=0.000000 test=2.000000
correct:  2
correct:  0
batch 860: train=2.000000 test=0.000000
correct:  1
correct:  1
batch 870: train=1.000000 test=1.000000
correct:  2
correct:  0
batch 880: train=2.000000 test=0.000000
correct:  0
correct:  1
batch 890: train=0.000000 test=1.000000
correct:  1
correct:  2
batch 900: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 910: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 920: train=1.000000 test=2.000000
correct:  2
correct:  0
batch 930: train=2.000000 test=0.000000
correct:  2
correct:  1
batch 940: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 950: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 960: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 970: train=1.000000 test=2.000000
correct:  1
correct:  1
batch 980: train=1.000000 test=1.000000
correct:  0
correct:  2
batch 990: train

correct:  2
correct:  2
batch 110: train=2.000000 test=2.000000
correct:  0
correct:  2
batch 120: train=0.000000 test=2.000000
correct:  1
correct:  2
batch 130: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 140: train=1.000000 test=2.000000
correct:  0
correct:  2
batch 150: train=0.000000 test=2.000000
correct:  2
correct:  2
batch 160: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 170: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 180: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 190: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 200: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 210: train=2.000000 test=1.000000
correct:  1
correct:  1
batch 220: train=1.000000 test=1.000000
correct:  2
correct:  0
batch 230: train=2.000000 test=0.000000
correct:  2
correct:  2
batch 240: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 250: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 260: train

correct:  1
correct:  1
batch 390: train=1.000000 test=1.000000
correct:  1
correct:  0
batch 400: train=1.000000 test=0.000000
correct:  1
correct:  1
batch 410: train=1.000000 test=1.000000
correct:  2
correct:  2
batch 420: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 430: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 440: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 450: train=2.000000 test=1.000000
correct:  0
correct:  1
batch 460: train=0.000000 test=1.000000
correct:  2
correct:  0
batch 470: train=2.000000 test=0.000000
correct:  2
correct:  2
batch 480: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 490: train=1.000000 test=1.000000
correct:  1
correct:  0
batch 500: train=1.000000 test=0.000000
correct:  1
correct:  2
batch 510: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 520: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 530: train=2.000000 test=2.000000
correct:  1
correct:  2
batch 540: train

correct:  1
correct:  2
batch 670: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 680: train=2.000000 test=2.000000
correct:  2
correct:  2
batch 690: train=2.000000 test=2.000000
correct:  2
correct:  2
batch 700: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 710: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 720: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 730: train=2.000000 test=2.000000
correct:  2
correct:  2
batch 740: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 750: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 760: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 770: train=2.000000 test=1.000000
correct:  2
correct:  0
batch 780: train=2.000000 test=0.000000
correct:  2
correct:  2
batch 790: train=2.000000 test=2.000000
correct:  2
correct:  2
batch 800: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 810: train=1.000000 test=1.000000
correct:  2
correct:  0
batch 820: train

correct:  2
batch 940: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 950: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 960: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 970: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 980: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 990: train=1.000000 test=2.000000
Evaluating...
Train accuracy: 0.755
Test accuracy: 0.745
Training...
correct:  0
correct:  1
batch 0: train=0.000000 test=1.000000
correct:  2
correct:  2
batch 10: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 20: train=2.000000 test=1.000000
correct:  1
correct:  1
batch 30: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 40: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 50: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 60: train=2.000000 test=1.000000
correct:  1
correct:  0
batch 70: train=1.000000 test=0.000000
correct:  1
correct:  2
batch 80: train=1.000000 test=2.

In [164]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=100
    inner_iters=20
    eval_inner_iters=20
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 10: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 20: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 30: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 40: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 50: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 60: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 70: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 80: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 90: train=2.000000 test=1.000000
Evaluating...
Train accuracy: 0.685
Test accuracy: 0.69
Training...
correct:  1
correct:  2
batch 0: train=1.000000 test=2.000000
correct:  1
correct:  2
batch 10: train=1.000000 test=2.000000
correct:  0
correct:  0
batch 20: train=0.000000 test=0.000000
correct:  0
correct:  1
batch 30: train=0.000000 test=1.000000
correct:  1
correct:  1
batch 40: train=

In [165]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=100
    inner_iters=50
    eval_inner_iters=50
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  0
correct:  1
batch 0: train=0.000000 test=1.000000
correct:  2
correct:  0
batch 10: train=2.000000 test=0.000000
correct:  1
correct:  0
batch 20: train=1.000000 test=0.000000
correct:  2
correct:  1
batch 30: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 40: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 50: train=2.000000 test=1.000000
correct:  1
correct:  1
batch 60: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 70: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 80: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 90: train=2.000000 test=2.000000
Evaluating...
Train accuracy: 0.755
Test accuracy: 0.645
Training...
correct:  2
correct:  2
batch 0: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 10: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 20: train=2.000000 test=2.000000
correct:  1
correct:  0
batch 30: train=1.000000 test=0.000000
correct:  2
correct:  1
batch 40: train

In [166]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)
    meta_iters=1000
    inner_iters=50
    eval_inner_iters=50
    num_shots=4
    test_accs = []
    train_accs = []
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        train_acc = reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Train accuracy: ' + str(train_acc))
        test_acc = reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)
        print('Test accuracy: ' + str(test_acc))
        train_accs.append(train_acc)
        test_accs.append(test_acc)
print(train_accs)
print(test_accs)

Training...
correct:  0
correct:  1
batch 0: train=0.000000 test=1.000000
correct:  0
correct:  1
batch 10: train=0.000000 test=1.000000
correct:  2
correct:  1
batch 20: train=2.000000 test=1.000000
correct:  2
correct:  1
batch 30: train=2.000000 test=1.000000
correct:  1
correct:  1
batch 40: train=1.000000 test=1.000000
correct:  2
correct:  2
batch 50: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 60: train=2.000000 test=1.000000
correct:  1
correct:  2
batch 70: train=1.000000 test=2.000000
correct:  2
correct:  2
batch 80: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 90: train=1.000000 test=1.000000
correct:  2
correct:  2
batch 100: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 110: train=1.000000 test=1.000000
correct:  0
correct:  2
batch 120: train=0.000000 test=2.000000
correct:  1
correct:  1
batch 130: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 140: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 150: trai

correct:  2
batch 270: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 280: train=2.000000 test=1.000000
correct:  1
correct:  0
batch 290: train=1.000000 test=0.000000
correct:  1
correct:  2
batch 300: train=1.000000 test=2.000000
correct:  2
correct:  1
batch 310: train=2.000000 test=1.000000
correct:  2
correct:  0
batch 320: train=2.000000 test=0.000000
correct:  2
correct:  2
batch 330: train=2.000000 test=2.000000
correct:  2
correct:  1
batch 340: train=2.000000 test=1.000000
correct:  2
correct:  2
batch 350: train=2.000000 test=2.000000
correct:  1
correct:  1
batch 360: train=1.000000 test=1.000000
correct:  1
correct:  2
batch 370: train=1.000000 test=2.000000
correct:  1
correct:  1
batch 380: train=1.000000 test=1.000000
correct:  1
correct:  1
batch 390: train=1.000000 test=1.000000
correct:  2
correct:  1
batch 400: train=2.000000 test=1.000000
correct:  1
correct:  1
batch 410: train=1.000000 test=1.000000
correct:  2
correct:  2
batch 420: train=2.000000 te

KeyboardInterrupt: 

In [ ]:
for _ in range(10):
    model = reptile.model.NNClassifier(2, 6)

    inner_iters=1
    eval_inner_iters=10
    num_shots=4
    with tf.Session() as sess:
        print('Training...')
        reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                            meta_iters=1, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
        print('Evaluating...')
        print('Train accuracy: ' + str(reptile.eval.evaluate(
            sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))
        print('Test accuracy: ' + str(reptile.eval.evaluate(
            sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))

In [158]:
model = reptile.model.NNClassifier(2, 6)

meta_iters=10000
inner_iters=10
eval_inner_iters=10
num_shots=4
with tf.Session() as sess:
    print('Training...')
    reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                        meta_iters=meta_iters, meta_step_size=0.1, inner_iters=inner_iters, num_shots=num_shots)
    print('Evaluating...')
    print('Train accuracy: ' + str(reptile.eval.evaluate(
        sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))
    print('Test accuracy: ' + str(reptile.eval.evaluate(
        sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters, num_shots=num_shots)))

Training...


KeyboardInterrupt: 

In [89]:
model = reptile.model.NNClassifier(2, 6)

inner_iters=20
eval_inner_iters=20
with tf.Session() as sess:
    print('Training...')
    reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                        meta_iters=10, meta_step_size=0.1, inner_iters=inner_iters)
    print('Evaluating...')
    print('Train accuracy: ' + str(reptile.eval.evaluate(
        sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters)))
    print('Test accuracy: ' + str(reptile.eval.evaluate(
        sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters)))


Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
Evaluating...
Train accuracy: 0.77
Test accuracy: 0.74


In [93]:
model = reptile.model.NNClassifier(2, 6)

inner_iters=20
eval_inner_iters=40
with tf.Session() as sess:
    print('Training...')
    reptile.train.train(sess, model, train_set, test_set, os.path.join(cur_dir, 'checkpoints'), 
                        meta_iters=1, meta_step_size=0.1, inner_iters=inner_iters)
    print('Evaluating...')
    print('Train accuracy: ' + str(reptile.eval.evaluate(
        sess, model, train_set, num_samples=100, eval_inner_iters=eval_inner_iters)))
    print('Test accuracy: ' + str(reptile.eval.evaluate(
        sess, model, test_set, num_samples=100, eval_inner_iters=eval_inner_iters)))

Training...
correct:  0
correct:  0
batch 0: train=0.000000 test=0.000000
Evaluating...
Train accuracy: 0.89
Test accuracy: 0.71


Training...
correct:  1
correct:  1
batch 0: train=1.000000 test=1.000000
Evaluating...


ValueError: Cannot take a larger sample than population when 'replace=False'

In [97]:
score_lf = [0.52117, 0.52180, 0.521830, 0.52207]
score_lfg = [0.52192, 0.52229, 0.52252, 0.52205]
print('score_lf: {0:.5f}+-{1:.5f}'.format(np.mean(score_lf), np.std(score_lf)))
print('score_lfg: {0:.5f}+-{1:.5f}'.format(np.mean(score_lfg), np.std(score_lfg)))

score_lf: 0.52172+-0.00033
score_lfg: 0.52220+-0.00023


In [48]:
0.52172+0.00033

0.52205

In [49]:
0.52220-0.00023

0.52197